# 🎯 LIDAR Grasp Detection v11

## ใหม่ใน v11
| Feature | Description |
|---------|-------------|
| **📏 TF-Luna LIDAR** | วัดความสูงแม่นยำ ไม่ขึ้นกับแสง |
| **🚀 Direct Z Calculation** | คำนวณ Z จาก LIDAR โดยตรง |
| **🎯 PCA Grasp** | ใช้ PCA วิเคราะห์ grasp เหมือน v10 |

## 🔌 Wiring TF-Luna + ESP32

```
TF-Luna          ESP32
────────         ─────
VCC (5V)    →    5V
GND         →    GND  
TX          →    GPIO16 (RX2)
RX          ←    GPIO17 (TX2)
```

⚠️ **สำคัญ:** TF-Luna ใช้ไฟ 5V (ไม่ใช่ 3.3V)

## 1️⃣ Imports

In [1]:
import cv2
import numpy as np
import time
import socket
import serial
from ultralytics import YOLO

print("✓ Imports")

✓ Imports


## 2️⃣ Hardware Configuration

In [2]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

print("✓ Hardware config")

✓ Hardware config


## 3️⃣ Configuration

In [3]:
# === CALIBRATED VALUES ===
PIXELS_PER_MM = 2.7703

# === Robot R Rotation ===
ROBOT_R_OFFSET = -25.55

# === Z Heights ===
Z_FLOOR = -64
Z_MEASURE = 120   # ความสูงที่ใช้วัด LIDAR (ต้องสูงพอให้ LIDAR มองเห็น >200mm)

# === LIDAR Configuration ===
LIDAR_TO_GRIPPER_OFFSET = 60  # mm (จาก LIDAR ถึงปลาย gripper ตอน Z=-64)
LIDAR_TIMEOUT = 1.0           # seconds

# === Drop Position ===
DROP_POS = (-253.07, 115.17, -17.07, -62.78)

# === Gripper ===
GRIPPER_SERVO_OPEN_ANGLE = 22
GRIPPER_SERVO_CLOSE_ANGLE = 96
GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_MIN_WIDTH_MM = 0
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

# === Detection ===
MIN_OBJECT_AREA = 1000
YOLO_CONFIDENCE = 0.25

print("✓ Configuration (v11 with LIDAR)")

✓ Configuration (v11 with LIDAR)


## 4️⃣ Load Models

In [4]:
print("Loading YOLOv8...")
yolo_model = YOLO('yolov8n.pt')
print("✅ YOLO loaded")

Loading YOLOv8...
✅ YOLO loaded


## 5️⃣ Classes Definition

In [5]:
class SmartGripperController:
    """Gripper + LIDAR Controller (v11)"""
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper+LIDAR on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: 
            self.serial.close()
    
    def send_command(self, cmd):
        if self.serial:
            self.serial.reset_input_buffer()
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {width_mm:.1f}mm - {GRIPPER_GRIP_MARGIN_MM}mm = {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None
    
    # ==================== LIDAR Functions ====================
    def read_lidar(self, samples=3):
        """Read LIDAR distance in mm (average of samples)"""
        if not self.serial:
            return None
        
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            
            start = time.time()
            while time.time() - start < LIDAR_TIMEOUT:
                if self.serial.in_waiting > 0:
                    response = self.serial.readline().decode().strip()
                    if response.startswith("LIDAR:") and "ERR" not in response:
                        try:
                            dist = int(response.split(":")[1])
                            readings.append(dist)
                        except:
                            pass
                        break
            time.sleep(0.05)
        
        if readings:
            return int(np.median(readings))
        return None

print("✓ Gripper+LIDAR class")

✓ Gripper+LIDAR class


In [6]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ Robot class")

✓ Robot class


In [7]:
class ObjectDetector:
    """v11: YOLO + Contour Detection (No Depth Model)"""
    
    def __init__(self, yolo_model, pixels_per_mm):
        self.yolo = yolo_model
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        results = self.yolo(frame, conf=YOLO_CONFIDENCE, verbose=False)
        
        for r in results:
            for box in r.boxes:
                x1,y1,x2,y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                
                roi = frame[y1:y2, x1:x2]
                if roi.size == 0: continue
                
                gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    cnt = max(contours, key=cv2.contourArea)
                    cnt = cnt + np.array([x1, y1])
                    rect = cv2.minAreaRect(cnt)
                    cx, cy = int(rect[0][0]), int(rect[0][1])
                    
                    objects.append({
                        'bbox': (x1, y1, x2-x1, y2-y1),
                        'center': (cx, cy),
                        'rect': rect,
                        'rect_size': rect[1],
                        'rect_angle': rect[2],
                        'contour': cnt,
                        'conf': conf,
                        'area': cv2.contourArea(cnt)
                    })
        
        if not objects:
            objects = self._edge_detect(frame)
        
        return objects
    
    def _edge_detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(cv2.GaussianBlur(gray, (5,5), 0), 50, 150)
        edges = cv2.dilate(edges, np.ones((3,3), np.uint8), iterations=2)
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_OBJECT_AREA:
                hull = cv2.convexHull(cnt)
                rect = cv2.minAreaRect(hull)
                x,y,w,h = cv2.boundingRect(hull)
                objects.append({
                    'bbox': (x,y,w,h),
                    'center': (x+w//2, y+h//2),
                    'rect': rect,
                    'rect_size': rect[1],
                    'rect_angle': rect[2],
                    'contour': hull,
                    'area': area
                })
        return sorted(objects, key=lambda o: o['area'], reverse=True)

print("✓ Object Detector class")

✓ Object Detector class


In [8]:
class PCAGraspSelector:
    """v11: PCA-based Grasp Selection"""
    
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        """Analyze object with PCA"""
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback_analysis(obj)
        
        # PCA Analysis
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]
        
        major_axis = eigenvectors[:, 0]
        minor_axis = eigenvectors[:, 1]
        
        angle_major = np.degrees(np.arctan2(major_axis[1], major_axis[0]))
        
        projections = np.dot(pts_centered, minor_axis)
        width_px = np.max(projections) - np.min(projections)
        width_mm = width_px / self.ppm
        
        proj_major = np.dot(pts_centered, major_axis)
        length_px = np.max(proj_major) - np.min(proj_major)
        length_mm = length_px / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        grasp_camera_angle = self._normalize_angle(angle_major + 90)
        
        grasps = []
        
        # Primary: Narrow side grasp
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': width_mm,
                'camera_angle': grasp_camera_angle,
                'score': 1.0,
                'type': 'PCA_narrow',
                'axis_info': {'major': major_axis, 'minor': minor_axis}
            })
        
        # Alternative: Long side grasp
        if length_mm <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': length_mm,
                'camera_angle': self._normalize_angle(angle_major),
                'score': 0.5,
                'type': 'PCA_long',
                'axis_info': None
            })
        
        if not grasps:
            return self._fallback_analysis(obj)
        
        return sorted(grasps, key=lambda g: g['score'], reverse=True)
    
    def _fallback_analysis(self, obj):
        rect = obj.get('rect')
        if rect is None:
            return []
        
        (cx, cy), (w, h), angle = rect
        cx, cy = int(cx), int(cy)
        
        if w < h:
            grip_w = w / self.ppm
            grip_a = angle + 90
        else:
            grip_w = h / self.ppm
            grip_a = angle
        
        grasps = []
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            grasps.append({
                'center': (cx, cy),
                'width_mm': grip_w,
                'camera_angle': self._normalize_angle(grip_a),
                'score': 0.6,
                'type': 'rect_fallback',
                'axis_info': None
            })
        
        return grasps
    
    def _normalize_angle(self, angle):
        while angle > 90: angle -= 180
        while angle < -90: angle += 180
        return angle

print("✓ PCA Grasp Selector")

✓ PCA Grasp Selector


## 6️⃣ Initialize Components

In [9]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = ObjectDetector(yolo_model, PIXELS_PER_MM)
grasp_selector = PCAGraspSelector(PIXELS_PER_MM)
print("✓ Components initialized (v11 LIDAR)")

✓ Components initialized (v11 LIDAR)


---
# 🧪 TEST LIDAR (No Robot)
---

In [10]:
print("="*60)
print("🧪 TEST LIDAR READING")
print("="*60)

gripper.connect()

print("\nReading LIDAR 10 times...")
for i in range(10):
    dist = gripper.read_lidar()
    if dist:
        print(f"  [{i+1}] Distance: {dist} mm ({dist/10:.1f} cm)")
    else:
        print(f"  [{i+1}] LIDAR: Error reading")
    time.sleep(0.3)

gripper.disconnect()
print("\n✅ LIDAR test complete")

🧪 TEST LIDAR READING
✅ Gripper+LIDAR on COM9

Reading LIDAR 10 times...
  [1] Distance: 290 mm (29.0 cm)
  [2] Distance: 290 mm (29.0 cm)
  [3] Distance: 290 mm (29.0 cm)
  [4] Distance: 290 mm (29.0 cm)
  [5] Distance: 290 mm (29.0 cm)
  [6] Distance: 290 mm (29.0 cm)
  [7] Distance: 290 mm (29.0 cm)
  [8] Distance: 290 mm (29.0 cm)
  [9] Distance: 290 mm (29.0 cm)
  [10] Distance: 290 mm (29.0 cm)

✅ LIDAR test complete


---
# 🧪 TEST GRASP MODEL (No Robot)
---

In [11]:
robot.home()

🤖 HOME...


In [12]:
print("="*60)
print("🧪 TEST GRASP MODEL (No Robot)")
print("   Click=Select Object | Q=Quit")
print("="*60)

test_selected = None
test_grasps = []

def test_mouse_cb(event, x, y, flags, param):
    global test_selected, test_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in param.get('objects', []):
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                test_selected = obj
                test_grasps = grasp_selector.analyze_object(obj)
                if test_grasps:
                    g = test_grasps[0]
                    print(f"\n📦 Selected: W={g['width_mm']:.1f}mm Angle={g['camera_angle']:.1f}°")
                    print(f"   Type: {g['type']}")
                break

cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Test v11')
callback_data = {'objects': []}
cv2.setMouseCallback('Test v11', test_mouse_cb, callback_data)

def draw_grasps(frame, grasps, selected):
    for g in grasps:
        cx, cy = g['center']
        angle = g['camera_angle']
        is_sel = (selected and g == selected)
        color = (0,0,255) if is_sel else ((0,255,0) if g['score']>=0.7 else (0,255,255))
        thick = 3 if is_sel else 2
        
        length = 40
        dx = int(length * np.cos(np.radians(angle)))
        dy = int(length * np.sin(np.radians(angle)))
        cv2.line(frame, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
        cv2.circle(frame, (cx, cy), 5, color, -1)
        
        grip_half = int(g['width_mm'] * PIXELS_PER_MM / 2)
        perp_angle = angle + 90
        px = int(grip_half * np.cos(np.radians(perp_angle)))
        py = int(grip_half * np.sin(np.radians(perp_angle)))
        cv2.line(frame, (cx+px-dx//2, cy+py-dy//2), (cx+px+dx//2, cy+py+dy//2), color, 2)
        cv2.line(frame, (cx-px-dx//2, cy-py-dy//2), (cx-px+dx//2, cy-py+dy//2), color, 2)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected = detector.detect(frame)
    callback_data['objects'] = detected
    
    display = frame.copy()
    for obj in detected:
        x,y,w,h = obj['bbox']
        is_sel = (test_selected and obj['center'] == test_selected['center'])
        color = (0,0,255) if is_sel else (0,255,0)
        cv2.rectangle(display, (x,y), (x+w,y+h), color, 2)
    
    if test_selected and test_grasps:
        draw_grasps(display, test_grasps, test_grasps[0])
    
    cv2.rectangle(display, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(display, f"v11 LIDAR | Obj:{len(detected)} | Click | Q=Quit",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    cv2.imshow('Test v11', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        test_selected = None
        test_grasps = []

cap.release()
cv2.destroyAllWindows()
print("\n✅ Test complete")

🧪 TEST GRASP MODEL (No Robot)
   Click=Select Object | Q=Quit

✅ Test complete


---
# 🤖 FULL ROBOT PICK WITH LIDAR
---

In [20]:
print("="*60)
print("🤖 Connecting to Robot and Gripper+LIDAR...")
print("="*60)

gripper.connect()
robot.connect(ROBOT_IP)
robot.home()

print("✅ Ready!")

🤖 Connecting to Robot and Gripper+LIDAR...
✅ Gripper+LIDAR on COM9
✅ Robot connected!
🤖 HOME...
✅ Ready!


In [14]:
# Test gripper connection
import serial
import time

ser = serial.Serial('COM9', 115200, timeout=2)
time.sleep(4)

# Test open
ser.write(b'O\n')
time.sleep(10)
print(ser.readline().decode().strip())

# Test close
ser.write(b'C\n')
time.sleep(10)
print(ser.readline().decode().strip())

ser.close()

SerialException: could not open port 'COM9': PermissionError(13, 'Access is denied.', None, 5)

In [22]:
gripper.open_for_object(10)

🦾 Open: 15.0mm (77°)


In [17]:
robot.home()

🤖 HOME...


In [23]:
selected_object = None
selected_grasp = None
current_grasps = []
detected_objects = []

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps
    if event == cv2.EVENT_LBUTTONDOWN:
        for g in current_grasps:
            gx, gy = g['center']
            if abs(x-gx) < 20 and abs(y-gy) < 20:
                selected_grasp = g
                robot_r = robot.camera_angle_to_robot_r(g['camera_angle'])
                print(f"\n🎯 Grasp: W={g['width_mm']:.1f}mm → R={robot_r:.1f}°")
                return
        for obj in detected_objects:
            bx,by,bw,bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    robot_r = robot.camera_angle_to_robot_r(selected_grasp['camera_angle'])
                    print(f"\n📦 Object: {len(current_grasps)} grasps")
                    print(f"   Best: W={selected_grasp['width_mm']:.1f}mm R={robot_r:.1f}°")
                break

def pick_with_lidar(obj, grasp):
    """v11: Pick using LIDAR for Z measurement"""
    cx, cy = grasp['center']
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    robot_r = robot.camera_angle_to_robot_r(camera_angle)
    
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    
    print(f"\n🤖 Pick (LIDAR): W={grip_w:.1f}mm R={robot_r:.1f}°")
    print(f"   Type: {grasp['type']}")
    
    # 1. Safe position
    print("🔄 Safe position...")
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    
    # 2. Open gripper
    gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(2.5)
    
    # 3. Move to measure position (above object)
    print(f"📏 Moving to Z_MEASURE={Z_MEASURE}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_MEASURE, robot_r, 2)
    
    # 4. Read LIDAR
    lidar_dist = gripper.read_lidar(samples=5)
    if lidar_dist is None:
        print("❌ LIDAR read failed! Aborting.")
        robot.home()
        return
    
    print(f"📏 LIDAR: {lidar_dist} mm")
    
    # 5. Calculate Z_GRASP
    z_grasp = Z_MEASURE - lidar_dist + LIDAR_TO_GRIPPER_OFFSET
    z_grasp = max(Z_FLOOR, z_grasp)
    
    print(f"📍 Z_GRASP = {Z_MEASURE} - {lidar_dist} + {LIDAR_TO_GRIPPER_OFFSET} = {z_grasp:.1f}")
    
    # 6. Go directly to grasp position
    robot.move_to_and_wait(robot_x, robot_y, z_grasp, robot_r, 2)
    
    # 7. Grip
    gripper.grip_object(grip_w - 8.5)
    time.sleep(2.5)
    
    # 8. Go directly to drop
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    
    # 9. Release
    gripper.release()
    time.sleep(2.5)
    robot.home()
    print("✅ Complete!")

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('Pick v11 LIDAR')
cv2.setMouseCallback('Pick v11 LIDAR', mouse_callback)

print("="*60)
print("🎯 PICK v11 (LIDAR-based)")
print("Click=Select | SPACE=Execute | H=Home | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected_objects = detector.detect(frame)
    
    display = frame.copy()
    for obj in detected_objects:
        x,y,w,h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        color = (0,0,255) if is_sel else (0,255,0)
        cv2.rectangle(display, (x,y), (x+w,y+h), color, 2)
    
    if selected_object and current_grasps:
        draw_grasps(display, current_grasps, selected_grasp)
    
    cv2.rectangle(display, (0,0), (640,35), (30,30,30), -1)
    cv2.putText(display, f"v11 LIDAR | Obj:{len(detected_objects)} | Click | SPACE | H | Q",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255), 1)
    
    if selected_grasp:
        robot_r = robot.camera_angle_to_robot_r(selected_grasp['camera_angle'])
        cv2.putText(display, f"[{selected_grasp['type']}: W={selected_grasp['width_mm']:.1f}mm R={robot_r:.0f} - SPACE]",
                   (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
    
    cv2.imshow('Pick v11 LIDAR', display)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_with_lidar(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []

cap.release()
cv2.destroyAllWindows()
gripper.disconnect()

🎯 PICK v11 (LIDAR-based)
Click=Select | SPACE=Execute | H=Home | Q=Quit

📦 Object: 2 grasps
   Best: W=29.6mm R=-19.8°

🤖 Pick (LIDAR): W=29.6mm R=-19.8°
   Type: PCA_narrow
🔄 Safe position...
   → JointMovJ(0,0,0,0)
🦾 Open: 54.0mm (22°)
📏 Moving to Z_MEASURE=120...
   → MovJ(14.483114712312023,69.12085898453589,120,-19.766358843322767)
📏 LIDAR: 220 mm
📍 Z_GRASP = 120 - 220 + 60 = -40.0
   → MovJ(14.483114712312023,69.12085898453589,-40,-19.766358843322767)
🦾 Grip: 21.1mm - 5mm = 16.1mm (76°)
   → MovJ(-253.07,115.17,-17.07,-62.78)
🤖 HOME...
✅ Complete!


In [ ]:
ROBOT_R_OFFSET = -25.55
